In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.reproduction import major_oxides
from sklearn.metrics import mean_squared_error
from lib import full_flow_dataloader
import mlflow
import mlflow.keras
import numpy as np
import datetime
import os
from pathlib import Path

os.environ["KERAS_BACKEND"] = "torch"

import torch
import keras


torch.manual_seed(42)
np.random.seed(42)

2024-04-22 10:35:26.209360: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 10:35:27.335100: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:

import torch.nn as nn
import torch.optim as optim

# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
import numpy as np
from pathlib import Path
import torch
from torch.utils.data import Dataset, DataLoader

edr_path = Path("/home/christian/projects/p9/baseline/data/calib_edr/calib_2015")

class CSVDataset(Dataset):
    def __init__(self, directory_path, composition_data):
        self.csv_files = []
        self.targets = []
        for directory in directory_path.iterdir():
            if directory.is_dir():
                files = [f for f in directory.glob('*.csv')]
                # Assuming get_composition_for_sample returns a DataFrame
                target = composition_data.get_composition_for_sample(directory.name)[major_oxides].values.flatten().astype(np.float32)
                if target.size > 0:  # Check if target array is not empty
                    self.csv_files.extend(files)
                    # Extend the targets list with the same target for each file in the directory
                    self.targets.extend([target] * len(files))

    def __len__(self):
        return len(self.csv_files)

    def __getitem__(self, idx):
        file_path = self.csv_files[idx]
        data = np.loadtxt(file_path, skiprows=29, delimiter=',')
        target = self.targets[idx]
        return torch.tensor(data, dtype=torch.float32), torch.tensor(target, dtype=torch.float32)

# Usage example:
# Assuming 'composition_data' is an instance of CompositionData
# dataset = CSVDataset(edr_path, composition_data)
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
# for data, targets in dataloader:
#     # Here you would integrate with your PyTorch model training loop
#     optimizer.zero_grad()
#     outputs = model(data)
#     loss = loss_function(outputs, targets)
#     loss.backward()
#     optimizer.step()


In [5]:
from lib.config import AppConfig
from lib.data_handling import CompositionData

config = AppConfig()

composition_data = CompositionData(config.composition_data_path)
dataset = CSVDataset(edr_path, composition_data)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)

In [6]:
from keras.models import Model
from keras.layers import Input, Conv1D, BatchNormalization, ReLU, Subtract, Dense, Flatten, Add

D=20

def create_spectral_cnn(input_shape, num_chemical_elements):
    # Module 1
    input_layer = Input(shape=input_shape)
    x = input_layer
    for _ in range(D):  # Assuming D is the number of convolutional layers in Module 1
        conv = Conv1D(filters=64, kernel_size=3, padding='same')(x)
        norm = BatchNormalization()(conv)
        x = ReLU()(norm)

    # Module 2 (Residual connection)
    # Adding a residual connection to help with gradient flow
    residual = Conv1D(filters=64, kernel_size=3, padding='same')(x)
    x = Add()([x, residual])

    # Flattening the output of Module 2 to feed into a dense network
    x = Flatten()(x)

    # Module 3 (Regression for chemical content)
    outputs = []
    for _ in range(num_chemical_elements):
        outputs.append(Dense(units=1)(x))

    model = Model(inputs=input_layer, outputs=outputs)
    return model

# Assuming we have 5500 features in the input and we want to predict 8 chemical elements
input_shape = (6444, 50)  # Features, Channels
num_chemical_elements = 8
model = create_spectral_cnn(input_shape, num_chemical_elements)

model.compile(optimizer='adam', loss='mse')  # Using mean squared error for regression tasks
model.summary()

2024-04-22 10:35:31.737809: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 10:35:31.738389: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 6444, 50)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 6444, 64)  │      9,664 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 6444, 64)  │        256 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 6444, 64)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 6444, 64)  │     12,352 │ re_lu[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 6444, 64)  │        256 │ conv1d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 6444, 64)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 6444, 64)  │     12,352 │ re_lu_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 6444, 64)  │        256 │ conv1d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 6444, 64)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 6444, 64)  │     12,352 │ re_lu_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 6444, 64)  │        256 │ conv1d_3[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_3 (ReLU)      │ (None, 6444, 64)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 6444, 64)  │     12,352 │ re_lu_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 6444, 64)  │        256 │ conv1d_4[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_4 (ReLU)      │ (None, 6444, 64)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 6444, 64)  │     12,352 │ re_lu_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 6444, 64)  │        256 │ conv1d_5[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_5 (ReLU)      │ (None, 6444, 64)  │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_6 (Conv1D)   │ (None, 6444, 64)  │     12,352 │ re_lu_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 6444, 64)  │        256 │ conv1d_6[0][0]    │
│ (BatchNormalizatio… │                   │            │                 

 Total params: 3,561,160 (13.58 MB)

 Trainable params: 3,558,600 (13.57 MB)

 Non-trainable params: 2,560 (10.00 KB)

In [7]:
from keras.callbacks import EarlyStopping

mlflow.keras.autolog()

# Assuming 'dataset' is your CSVDataset and has been suitably converted to a format Keras can handle
# For demonstration, let's assume you have X_train, y_train prepared as NumPy arrays

# Define callbacks for early stopping and model checkpointing
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min'),
]

# Train the model
# history = model.fit(
#     dataset,
#     # validation_split=0.2,  # using 20% of data for validation
#     epochs=50,
#     batch_size=32,
#     callbacks=callbacks,
# )

# Load the best model saved by ModelCheckpoint
# model.load_weights('best_model.h5')

# Optionally, evaluate the model performance on a test set
# test_loss = model.evaluate(X_test, y_test)
# print(f"Test Loss: {test_loss}")


history = model.fit(dataloader, epochs=50, batch_size=32, callbacks=callbacks)
    


# for data, targets in dataloader:
#     # Here you would integrate with your PyTorch model training loop
#     optimizer.zero_grad()
#     outputs = model(data)
#     loss = loss_function(outputs, targets)
#     loss.backward()
#     optimizer.step()


2024/04/22 10:35:32 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of keras. If you encounter errors during autologging, try upgrading / downgrading keras to a supported version, or try upgrading MLflow.


2024/04/22 10:35:32 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ed5e4379604942be983136778af40103', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current keras workflow
2024/04/22 10:35:32 WARNING mlflow.keras.autologging: Unrecognized dataset type <class 'torch.utils.data.dataloader.DataLoader'>. Dataset logging skipped.


Epoch 1/50


/home/christian/projects/p9/baseline/venv/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:588: UserWarning: Gradients do not exist for variables ['kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


 1/77 ━━━━━━━━━━━━━━━━━━━━ 29:41 23s/step - loss: nan

KeyboardInterrupt: 